---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [ ]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [2]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    uni_file = open('university_towns.txt')
    uni_town_data = []
    for line in uni_file:
        edit_pos = line.find('[edit]')
        par_pos = line.find(' (')

        if edit_pos >=0:
            state = line[:edit_pos]        
        elif par_pos >=0:
            town = line[:par_pos]
            town = town.replace("\n", "")   
            uni_town_data.append([state, town])
        else:
            town = line
            town = town.replace("\n", "")
            uni_town_data.append([state, town])

    uni = pd.DataFrame(uni_town_data, columns=["State", "RegionName"])    
    return uni

In [ ]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    gdp_file = "gdplev.xls"
    gdp = pd.read_excel(gdp_file, skiprows=5, usecols=[4,6])
    gdp.columns = ['Period', 'GDP']
    gdp['change'] = gdp['GDP'].diff()
    gdp = gdp.drop(gdp.index[:214])


    eco_state = ['N']
    cur_state = 'N'

    for i, row in gdp.iterrows():
        if i in gdp.index[range(0,1)]:
            continue

        growth_last_quarter = gdp.loc[i-1]['change'] >=0
        growth_curr_quarter = row['change']>=0    

        if cur_state == 'N':
            if not growth_curr_quarter:
                if not growth_last_quarter:
                    cur_state = 'R'
                    eco_state.append('SR')
                else:
                    eco_state.append('N')
            else:
                eco_state.append('N')

        elif cur_state == 'R':
            if growth_curr_quarter:
                if growth_last_quarter:
                    cur_state = 'N'
                    eco_state.append('ER')                
                else:
                    eco_state.append('R')

            else:
                eco_state.append('R')
        else:
            raise Exception('Invalid State found.')

    gdp['Eco Log'] = eco_state
    rec_starts = gdp[gdp['Eco Log'] == 'SR']['Period'].tolist()
    return rec_starts[0]

In [ ]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    gdp_file = "gdplev.xls"
    gdp = pd.read_excel(gdp_file, skiprows=5, usecols=[4,6])
    gdp.columns = ['Period', 'GDP']
    gdp['change'] = gdp['GDP'].diff()
    gdp = gdp.drop(gdp.index[:214])


    eco_state = ['N']
    cur_state = 'N'

    for i, row in gdp.iterrows():
        if i in gdp.index[range(0,1)]:
            continue

        growth_last_quarter = gdp.loc[i-1]['change'] >=0
        growth_curr_quarter = row['change']>=0    

        if cur_state == 'N':
            if not growth_curr_quarter:
                if not growth_last_quarter:
                    cur_state = 'R'
                    eco_state.append('SR')
                else:
                    eco_state.append('N')
            else:
                eco_state.append('N')

        elif cur_state == 'R':
            if growth_curr_quarter:
                if growth_last_quarter:
                    cur_state = 'N'
                    eco_state.append('ER')                
                else:
                    eco_state.append('R')

            else:
                eco_state.append('R')
        else:
            raise Exception('Invalid State found.')

    gdp['Eco Log'] = eco_state
    rec_end = gdp[gdp['Eco Log'] == 'ER']['Period'].tolist()      
    return rec_end[0]     

In [ ]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    gdp_file = "gdplev.xls"
    gdp = pd.read_excel(gdp_file, skiprows=5, usecols=[4,6])
    gdp.columns = ['Period', 'GDP']
    gdp['change'] = gdp['GDP'].diff()
    gdp = gdp.drop(gdp.index[:214])


    eco_state = ['N']
    cur_state = 'N'

    for i, row in gdp.iterrows():
        if i in gdp.index[range(0,1)]:
            continue

        growth_last_quarter = gdp.loc[i-1]['change'] >=0
        growth_curr_quarter = row['change']>=0    

        if cur_state == 'N':
            if not growth_curr_quarter:
                if not growth_last_quarter:
                    cur_state = 'R'
                    eco_state.append('SR')
                else:
                    eco_state.append('N')
            else:
                eco_state.append('N')

        elif cur_state == 'R':
            if growth_curr_quarter:
                if growth_last_quarter:
                    cur_state = 'N'
                    eco_state.append('ER')                
                else:
                    eco_state.append('R')

            else:
                eco_state.append('R')
        else:
            raise Exception('Invalid State found.')

    gdp['Eco Log'] = eco_state
    rec_end = gdp[gdp['Eco Log'] == 'ER']['Period'].tolist()
    recession = gdp[gdp['Eco Log'].isin(['SR','R'])]
    
    return recession.loc[recession['GDP'].argmin()]['Period']    

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    import pandas as pd
    zillow_file = "City_Zhvi_AllHomes.csv"
    zillow = pd.read_csv(zillow_file)

    zillow['State'].replace(states, inplace=True)

    quarters = ['q1', 'q2','q3','q4']
    period = ["{}{}".format(year, quarter) for year in range(2000, 2017) for quarter in quarters]
    period = period[: len(period)-1]


    zillow_header =["{}-{:02}".format(year,month) for year in range(2000,2017) for month in range(1,13)]
    zillow_header = zillow_header[:len(zillow_header)-4]
    zillow.drop(['RegionID', 'Metro', 'CountyName', 'SizeRank'], axis=1, inplace = 1)
    zillow = zillow.set_index(['State', 'RegionName'])
    zillow_drop = zillow.columns
    for i in range(len(period)):
        month_cols = zillow_header[i*3: min(len(zillow_header), i*3 +3)]
        zillow[period[i]] = zillow[month_cols].mean(axis=1)

    zillow.drop(zillow_drop,axis=1, inplace=1)
        
    return zillow

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    import scipy.stats as stats
    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()
    uni_town = get_list_of_university_towns()
    uni_town_list = uni_town['RegionName'].tolist()

    house_data = convert_housing_data_to_quarters()
    house_data['price change'] = house_data[recession_start] - house_data[recession_bottom]

    house_data = house_data.reset_index()
    uni_town = house_data[house_data['RegionName'].isin(uni_town_list)]
    reg_town = house_data[~house_data['RegionName'].isin(uni_town_list)]

    uni_town = uni_town.dropna()
    reg_town = reg_town.dropna()

    alpha=0.01
    h_test_equal = stats.ttest_ind(uni_town['price change'], reg_town['price change'])

    mu_uni = uni_town['price change'].mean()
    mu_reg = reg_town['price change'].mean()


    p_value = h_test_equal[1]
    better = "university town" if mu_uni < mu_reg else "non-university town"

    
    return (p_value < alpha, p_value, better)    